In [1]:
import torch
import torchvision

from torch import nn
from torchsummary import summary   #used for finding summary
from torch.utils.data import DataLoader  
import torchvision.datasets as datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable

import time
from progressbar import *
import os
import matplotlib.pyplot as plt 
import numpy as np

In [2]:
#Enable cuda if cuda is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device, "is available ")

cuda is available 


### Model Making 

In [15]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, 3, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
    
    def forward(self, data):
        return self.encoder(data)
    
class decoder(nn.Module):
    
    def __init__(self):
        super(decoder, self).__init__()
        self.decoder = nn.Sequential(
                nn.ConvTranspose2d(64, 128, 4,stride=1, padding = 2),
                nn.ReLU(),
                nn.ConvTranspose2d(128, 64, 4, stride = 2, padding = 1),
                nn.ReLU(),
                nn.ConvTranspose2d(64,3,4, stride=2, padding=1)
        )

    def forward(self, x):
        return self.decoder(x)


encoder_model = encoder().to(device)
decoder_model = decoder().to(device)
criterion = nn.MSELoss()

summary(encoder_model, (3,64,64))   #used for finding summary for encoder 
summary(decoder_model, (64,17,17))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 66, 66]             896
              ReLU-2           [-1, 32, 66, 66]               0
         MaxPool2d-3           [-1, 32, 33, 33]               0
            Conv2d-4           [-1, 64, 35, 35]          18,496
              ReLU-5           [-1, 64, 35, 35]               0
         MaxPool2d-6           [-1, 64, 17, 17]               0
Total params: 19,392
Trainable params: 19,392
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 3.73
Params size (MB): 0.07
Estimated Total Size (MB): 3.85
----------------------------------------------------------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1          [-1, 128, 16,

In [24]:
class vae(nn.Module):
    def __init__(self):
        super(vae, self).__init__()
        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)
        self.linear_dense = nn.Linear(17,8)
        
    def forward(self,data):
        print(data.shape)
        x = self.encoder.forward(data)
        x = self.linear_dense(x)
        return self.decoder.forward(x)

In [25]:
vae = vae().to(device)
summary(vae, (3,64,64))

torch.Size([2, 3, 64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 66, 66]             896
              ReLU-2           [-1, 32, 66, 66]               0
         MaxPool2d-3           [-1, 32, 33, 33]               0
            Conv2d-4           [-1, 64, 35, 35]          18,496
              ReLU-5           [-1, 64, 35, 35]               0
         MaxPool2d-6           [-1, 64, 17, 17]               0
            Linear-7            [-1, 64, 17, 8]             144
   ConvTranspose2d-8           [-1, 128, 16, 7]         131,200
              ReLU-9           [-1, 128, 16, 7]               0
  ConvTranspose2d-10           [-1, 64, 32, 14]         131,136
             ReLU-11           [-1, 64, 32, 14]               0
  ConvTranspose2d-12            [-1, 3, 64, 28]           3,075
Total params: 284,947
Trainable params: 284,947
Non-trainable params: 0
----